In this notebook, we will create and compare several models. Our target feature for this notebooks is LOG(VIEWS/SUBSCRIBER.) 


In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv(r"no_early_dates_90_days.csv")

In [3]:
features = ["popular_brand", "has_any_affiliate", "product", "budget", "self_ref", "acronym", "korean", "speed", "skills/teach", "skincare", "comparing_products", "prime_hour", "hashtags", "hasAdinTitle", "hasAdinText"]

#Create the target column $y$ here:


#We don't need a lot of the noise columns
df = df[ features + "y" ] 

TypeError: can only concatenate list (not "str") to list

In [ ]:
#Do an 80-20 Train Test Split Here. Never ever touch the testing set please!

In [ ]:
#Create the baseline model here

In [ ]:
#Creat the basic linear regression model here 

In [ ]:
#Create the basic linear regression model here with lasso regression. 

In [ ]:
#Create a model whose features include all interaction terms 

In [ ]:
#Create a model with all interaction terms and lasso regression 

In [ ]:
#Do cross-validation to compare all models 

In [ ]:
#Final interpretation 